In [63]:
!pip install groq


In [64]:
from groq import Groq
import os
from getpass import getpass

In [65]:
api_key = getpass("from getpass import getpass")
os.environ["GROQ_API_KEY"] = api_key
client = Groq(api_key=os.environ["GROQ_API_KEY"])

from getpass import getpass··········


In [66]:

class ConversationManager:
    def __init__(self, summarize_every=3):
        self.history = []
        self.run_count = 0
        self.summarize_every = summarize_every

    def add_user_message(self, message):
        self.history.append({"role": "user", "content": message})
        self.run_count += 1
        self.truncate_by_turns(5)
        if self.run_count % self.summarize_every == 0:
            self.summarize_history()
        reply = self.generate_reply()
        self.history.append({"role": "assistant", "content": reply})
        return reply

    def generate_reply(self):
        context = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in self.history])
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": context}]
        )
        return response.choices[0].message.content

    def truncate_by_turns(self, n=5):
        self.history = self.history[-n:]

    def truncate_by_length(self, max_chars=300):
        text = ""
        truncated = []
        for msg in reversed(self.history):
            if len(text) + len(msg["content"]) > max_chars:
                break
            truncated.insert(0, msg)
            text += msg["content"]
        self.history = truncated

    def summarize_history(self):
        prompt = "Summarize this conversation in 2-3 sentences:\n"
        for msg in self.history:
            prompt += f"{msg['role'].capitalize()}: {msg['content']}\n"
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": prompt}]
        )
        summary = response.choices[0].message.content
        self.history = [{"role": "system", "content": f"Summary: {summary}"}]

    def get_history(self):
        return self.history

In [68]:
class ChatAnalyzerOptimized:
    def __init__(self, client, model="llama-3.1-8b-instant"):
        self.client = client
        self.model = model

    def analyze_messages(self, messages):
        system_prompt = """
        You are an assistant that classifies and extracts structured data from user messages.
        For each message, provide:
          - category: greeting, question, feedback, complaint, preference, other
          - extracted_data: Python dict with fields 'genre' and 'character_type'
        Respond only with a Python list of dictionaries, one per message:
        Example:
        [
          {"message": "Hi there!", "category": "greeting", "extracted_data": {"genre": "", "character_type": ""}},
          {"message": "I like action movies with strong lead characters.", "category": "preference", "extracted_data": {"genre": "action", "character_type": "strong lead"}}
        ]
        """
        user_prompt = "Messages:\n" + "\n".join([f"- {m}" for m in messages])
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        try:
            results = eval(response.choices[0].message.content)
            if not isinstance(results, list):
                results = []
        except:
            results = []
        return results

In [69]:
chat_schema = {
    "name": "string",
    "email": "string",
    "phone": "string",
    "location": "string",
    "age": "integer"
}

sample_chats = [
    "Hi, my name is Alice Johnson. I live in New York, I'm 28, my email is alice@example.com, and my phone is 555-1234.",
    "Hello, I'm Bob. Age 35. Email: bob123@gmail.com, Phone: 987-6543, Location: Los Angeles.",
    "Hey, this is Charlie, 22 years old, living in Chicago. You can reach me at charlie22@domain.com, phone 444-555-6666."
]

In [70]:
def extract_info_using_groq(chats):
    system_prompt = """
    You are an assistant that extracts structured information from user messages.
    The fields to extract are: name, email, phone, location, age.
    Respond in Python dict format only.
    """
    results = []
    for chat in chats:
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": chat}
            ]
        )
        try:
            data = eval(response.choices[0].message.content)
            extracted = {k: data.get(k, "") for k in chat_schema.keys()}
        except:
            extracted = {k: "" for k in chat_schema.keys()}
        results.append({"chat": chat, "extracted_data": extracted})
    return results

manager = ConversationManager(summarize_every=3)
analyzer = ChatAnalyzerOptimized(client)

conversation_messages = [
    "Hi, I’m looking for movie recommendations.",
    "I like action movies with strong lead characters.",
    "Any new releases this year?",
    "I didn’t like the last movie I watched."
]

for msg in conversation_messages:
    reply = manager.add_user_message(msg)
    print(f"User: {msg}")
    print(f"Assistant: {reply}\n")

print("=== Conversation History ===")
for msg in manager.get_history():
    print(msg)

analysis_results = analyzer.analyze_messages(conversation_messages)
print("\n=== Classification & Structured Extraction ===")
for r in analysis_results:
    print("Message:", r["message"])
    print("Category:", r["category"])
    print("Extracted Data:", r["extracted_data"])
    print("-" * 50)

print("\n=== Task 2: JSON Schema Extraction ===")
json_results = extract_info_using_groq(sample_chats)
for r in json_results:
    print("Chat:", r["chat"])
    print("Extracted Data:", r["extracted_data"])
    print("-" * 50)

User: Hi, I’m looking for movie recommendations.
Assistant: I'd be happy to help you with some movie recommendations. Do you have any specific preferences or genres you're interested in? For example, are you looking for:

- Action movies
- Romantic comedies
- Sci-fi or fantasy films
- Horror movies
- Classic films
- Recent releases
- Animated movies
- Something else?

Or maybe there's a particular theme or theme that you're in the mood for, like historical dramas, superhero movies, or documentaries.

Also, are you looking for something light-hearted and easy to watch, or something more complex and thought-provoking?

User: I like action movies with strong lead characters.
Assistant: Action movies with strong lead characters can be thrilling and engaging. Here are some recommendations that might interest you:

**Recent Releases:**

1. **Mad Max: Fury Road** (2015) - an adrenaline-fueled action film with a strong female lead in Charlize Theron's Imperator Furiosa.
2. **Wonder Woman** (20

In [ ]:
from google.colab import files

uploaded = files.upload()
import os
os.chdir("/content/chatsummaryandanalyser")
!mv /content/groq_chat_assignment.ipynb ./groq_chat_assignment.ipynb
!git add groq_chat_assignment.ipynb
!git commit -m "Update notebook with combined Task 1 and Task 2"
!git push origin main